In [1]:
from utils.PinConfig import PinConfig
import casadi
import pinocchio as pin

config_file = 'config/SimpleConfig.yaml'
robot, model, data, cmodel, cdata = PinConfig(config_file)
print(robot.q0)

cq = casadi.SX.sym('q', model.nq)

[0. 0. 0. 0. 0. 0.]


In [2]:
from utils.ObstacleSpheres import *
import numpy as np
config_file = 'config/Config.yaml'
obs = ObsConfig(config_file)
print(obs.robot_config)

obs.spheres_from_robot(q0= [0.0,0,0,0,0,0])  # Get initial robot obstacle spheres
obs.spheres_from_env()
print(obs.env_sphere_list)




{'pinocchio_config': {'urdf_path': '/home/angie/Projects/curobo/src/curobo/content/assets/robot/ur_description/ur5e.urdf', 'robex': None}, 'curobo_config': {'robot_cfg': {'kinematics': {'usd_path': 'robot/ur_description/ur5e.usd', 'usd_robot_root': '/robot', 'isaac_usd_path': '', 'usd_flip_joints': {}, 'usd_flip_joint_limits': [], 'urdf_path': 'robot/ur_description/ur5e.urdf', 'asset_root_path': 'robot/ur_description', 'base_link': 'base_link', 'ee_link': 'tool0', 'link_names': ['shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link'], 'lock_joints': None, 'extra_links': None, 'collision_link_names': None, 'collision_spheres': None, 'collision_sphere_buffer': 0.005, 'extra_collision_spheres': {}, 'self_collision_ignore': None, 'self_collision_buffer': None, 'use_global_cumul': True, 'mesh_link_names': ['shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link'], 'cspace': {'joint_names': ['shoulder_pan_joi

In [3]:
get_obstacle_distances = casadi.Function(
    'get_obstacle_distances', 
    [cq], 
    [casadi.vertcat(*obstacle_distances(obs.env_sphere_list, obs.robot_sphere_list, cmodel, cdata, cq))]
    )

In [4]:
nq = model.nq   # Number of joints
nv = model.nv   # Number of DOF
nx = nq + nv    # Number of states
cx = casadi.SX.sym("x", nx, 1)
ctau = casadi.SX.sym("tau", nv, 1)
cacc = cpin.aba(cmodel, cdata, cx[:nq], cx[nq:], ctau) # Forward dynamics

cq = casadi.SX.sym("x", model.nq, 1)


# Joint limits
lowerVelocityLimit = np.pi/2 * np.array([-1, -1, -1, -1, -1, -1])  # rad/s
upperVelocityLimit = np.pi/2 * np.array([1, 1, 1, 1, 1, 1])
lowerTorqueLimit = 100 * np.array([-1, -1, -1, -1, -1, -1]) 
upperTorqueLimit = 100 * np.array([1, 1, 1, 1, 1, 1])
print(model.upperPositionLimit)
print(model.lowerPositionLimit)

waypoints = np.array([[0, np.pi/4, 0, 0, 0, 0], 
                      [-np.pi/6, np.pi/3, np.pi/6, -np.pi/6, -np.pi/3, -np.pi/4],
                      [-np.pi/4, np.pi/4, np.pi/3, 0, -np.pi/6, -np.pi/2]])
q0 = waypoints[0]
robot.q0 = q0
qT = waypoints[-1]
tau0 = pin.rnea(model, data, q0, np.zeros(nv), np.zeros(nv))
tauT = pin.rnea(model, data, qT, np.zeros(nv), np.zeros(nv))
print(tau0)
print(tauT)

[6.28318531 6.28318531 3.14159265 6.28318531 6.28318531 6.28318531]
[-6.28318531 -6.28318531 -3.14159265 -6.28318531 -6.28318531 -6.28318531]
[-2.25158203e-15 -8.52573478e+01 -2.38834632e+01  1.62120800e-01
  0.00000000e+00  0.00000000e+00]
[-4.75225193e-16 -5.23511289e+01  9.02275572e+00  2.21461131e-01
  0.00000000e+00  0.00000000e+00]


In [5]:
from utils.Lagrange import *
from utils.LGL_collocation import LGL_collocation
from utils.LocalChart import *

Ntp = 10
Np = waypoints.shape[0]-1  # Number of phase (dersired waypoints: N+1)
T = Np*Ntp  # Number of total collocation points / optimization variables

LGL_Pts, LGL_Wts = Normalize_LGL_Pts_Wts(Ntp)
DL,_ = differentiation_matrix(Ntp, LGL_Pts)

LGL = LGL_collocation(Ntp, nq, nv)
StatesDiff, StateDiff, LocalsDiff, LocalDiff= LGL.getDiffFuncs()

Feq = casadi.Function(
    "Dyn",
    [cx, ctau],
    [casadi.vertcat(cx[nq:], cacc)],
)

In [6]:
opti = casadi.Opti()
# Optimization variables
var_xs = [opti.variable(nx) for _ in range(T)]
var_us = [opti.variable(nv) for _ in range(T)]
var_deltaT = [opti.variable(1) for _ in range(Np)]

# Constraints
for i in range(Np):
    opti.subject_to(var_deltaT[i] >= 0.0)

    for tp in range(Ntp):
        t = i*Ntp + tp
        # dynamics
        opti.subject_to(StatesDiff(casadi.vertcat(*var_xs[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[tp,:]) 
                        == Feq(var_xs[t], var_us[t]))
        
        # joint constraints
        opti.subject_to(var_xs[t][:nq] <= model.upperPositionLimit)
        opti.subject_to(var_xs[t][:nq] >= model.lowerPositionLimit)
        # joint velocity constraints
        opti.subject_to(var_xs[t][nq:] <= upperVelocityLimit)
        opti.subject_to(var_xs[t][nq:] >= lowerVelocityLimit)
        # torque constraints
        opti.subject_to(var_us[t] <= upperTorqueLimit)
        opti.subject_to(var_us[t] >= lowerTorqueLimit)

        # collision avoidance
        opti.subject_to(get_obstacle_distances(var_xs[t][:nq]) >= 0.05)

# waypoints constraints (N-1 middle waypoints)
for i in range(1, Np):
    opti.subject_to(var_xs[i*Ntp][:nq] == waypoints[i])

# continuity
for i in range(1, Np):
    opti.subject_to(var_xs[i*Ntp-1] == var_xs[i*Ntp])
    opti.subject_to(var_us[i*Ntp-1] == var_us[i*Ntp])

# continuity of derivatives
for i in range(Np-1):
    opti.subject_to(StatesDiff(casadi.vertcat(*var_xs[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[Ntp-1,:]) 
                    == StatesDiff(casadi.vertcat(*var_xs[(i+1)*Ntp: (i+2)*Ntp]), var_deltaT[i+1], DL[0,:]))
    
    opti.subject_to(LocalDiff(casadi.vertcat(*var_us[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[Ntp-1,:])
                   == LocalDiff(casadi.vertcat(*var_us[(i+1)*Ntp: (i+2)*Ntp]), var_deltaT[i], DL[0,:]))


# boundary conditions
opti.subject_to(var_xs[0][:nq] == q0)
opti.subject_to(var_xs[-1][:nq] == qT)
opti.subject_to(var_xs[0][nq:] == 0)
opti.subject_to(var_xs[-1][nq:] == 0)
#opti.subject_to(var_us[0] == tau0)
#opti.subject_to(var_us[-1] == tauT)

# Objective
totalcost = 0

R = np.eye(nv)
for i in range(Np):
    for tp in range(Ntp):
        var_xs_Phase = var_xs[i*Ntp: (i+1)*Ntp]
        dq = [dq_Phase[nq:] for dq_Phase in var_xs_Phase]
        ddq = StateDiff(casadi.vertcat(*dq), var_deltaT[i], DL[tp,:])
        totalcost += var_deltaT[i]/2 * LGL_Wts[tp] * casadi.mtimes([ddq.T, R, ddq])
        
    totalcost += var_deltaT[i]

opti.minimize(totalcost)
opti.solver("ipopt") 

for x in var_xs:
    opti.set_initial(x, np.concatenate([robot.q0, np.zeros(nv)]))
for u in var_us:
    opti.set_initial(u, tau0)
for t in var_deltaT:
    opti.set_initial(t, 1)




In [ ]:
try:
    sol = opti.solve_limited()
    sol_xs = [opti.value(var_x) for var_x in var_xs]
    sol_us = [opti.value(var_u) for var_u in var_us]
    sol_T = [opti.value(var_t) for var_t in var_deltaT]
except:
    print("ERROR in convergence, plotting debug info.")
    sol_xs = [opti.debug.value(var_x) for var_x in var_xs]
    sol_us = [opti.debug.value(var_u) for var_u in var_us]
